In [1]:
#https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
#https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

#영문->프랑스어 버전
#http://www.manythings.org/anki/fra-eng.zip
##영문->한국어는 학습이 잘 안됨...

!git clone https://github.com/jukyellow/artificial-intelligence-study.git

Cloning into 'artificial-intelligence-study'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 1026 (delta 154), reused 90 (delta 33), pack-reused 759
Receiving objects: 100% (1026/1026), 16.39 MiB | 8.31 MiB/s, done.
Resolving deltas: 100% (560/560), done.


In [2]:
cd artificial-intelligence-study/06_논문_study/01_seq2seq/kor-eng/

/content/artificial-intelligence-study/06_논문_study/01_seq2seq/kor-eng


In [3]:
'''
#Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
**Summary of the algorithm**
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
**Data download**
[English to French sentence pairs.
](http://www.manythings.org/anki/fra-eng.zip)
[Lots of neat sentence pairs datasets.
](http://www.manythings.org/anki/)
**References**
- [Sequence to Sequence Learning with Neural Networks
   ](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    ](https://arxiv.org/abs/1406.1078)
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [5]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
#data_path = 'fra-eng/fra.txt'
#data_path = 'kor.txt'
data_path = 'fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    #input_text, target_text = line.split('\t')
    lineToken = line.split('\t')
    input_text, target_text = lineToken[0:2]
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [6]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])
print('input_token_index:', len(input_token_index))
print('target_token_index:', len(target_token_index))

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

print('encoder_input_data:', np.array(encoder_input_data).shape)
print('decoder_input_data:', np.array(decoder_input_data).shape)
print('decoder_target_data:', np.array(decoder_target_data).shape)

input_token_index: 70
target_token_index: 93
encoder_input_data: (10000, 16, 70)
decoder_input_data: (10000, 59, 93)
decoder_target_data: (10000, 59, 93)


In [16]:
######################################################################################################################################
#https://tykimos.github.io/2018/09/14/ten-minute_introduction_to_sequence-to-sequence_learning_in_Keras/
#Keras RNN/LSTM의 특징
#return_state는 encoder의 출력과 내부 RNN 상태인 리스트를 반환하도록 RNN을 구성하는 인수입니다. 이는 encoder의 상태를 복구하는 데 사용합니다.
#inital_state는 RNN의 초기 상태를 지정하는 인수입니다. 초기 상태로 incoder를 decoder로 전달하는 데 사용합니다.
#return_sequences는 출력된 전체 시퀀스를 반환하도록 구성하는 인수(마지막 출력을 제외하곤 기본 동작)로 decoder에 사용합니다.
######################################################################################################################################

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
print('encoder:', encoder)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
print('encoder_outputs:', encoder_outputs)
print('state_h:', state_h)
print('state_c:', state_c)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

encoder: <keras.layers.recurrent.LSTM object at 0x7f5540263d68>
encoder_outputs: Tensor("lstm_3/TensorArrayReadV3:0", shape=(?, 256), dtype=float32)
state_h: Tensor("lstm_3/while/Exit_2:0", shape=(?, 256), dtype=float32)
state_c: Tensor("lstm_3/while/Exit_3:0", shape=(?, 256), dtype=float32)
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, 70)     0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None, 93)     0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 334848      input_5[0][0]                  

In [8]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 8000 samples, validate on 2000 samples
Epoch 1/100





8000/8000 [==============================] - 22s 3ms/step - loss: 1.1999 - acc: 0.7233 - val_loss: 1.1133 - val_acc: 0.6997
Epoch 2/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.8642 - acc: 0.7664 - val_loss: 0.8612 - val_acc: 0.7589
Epoch 3/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.6879 - acc: 0.8061 - val_loss: 0.7220 - val_acc: 0.7935
Epoch 4/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5992 - acc: 0.8257 - val_loss: 0.6622 - val_acc: 0.8065
Epoch 5/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5482 - acc: 0.8390 - val_loss: 0.6216 - val_acc: 0.8149
Epoch 6/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5134 - acc: 0.8486 - val_loss: 0.5872 - val_acc: 0.8243
Epoch 7/100
8000/8000 [=========

In [0]:

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    #print('states_value:', np.array(states_value).shape)
    #print('states_value:', states_value)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.
    #print('target_seq:', np.array(target_seq).shape)
    #print('target_seq:', target_seq)

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        #print('output_tokens:', output_tokens)
        #print('h:', h)
        #print('c:', c)
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        #print('sampled_token_index:', sampled_token_index)
        sampled_char = reverse_target_char_index[sampled_token_index]
        #print('sampled_char:', sampled_char)
        decoded_sentence += sampled_char
        #print('decoded_sentence:', decoded_sentence)

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [15]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.

    rand_pick = np.random.choice(10000, 1)
    print('rand_pick:', rand_pick)
    seq_index = int(rand_pick)

    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

rand_pick: [9188]
-
Input sentence: Turn on the TV.
Decoded sentence: Rouge ça.

rand_pick: [1250]
-
Input sentence: I feel old.
Decoded sentence: Je me sens mal.

rand_pick: [9701]
-
Input sentence: Are those yours?
Decoded sentence: Ellavont-il ?

rand_pick: [8736]
-
Input sentence: Shall we order?
Decoded sentence: Est-ce que je commence ?

rand_pick: [2113]
-
Input sentence: I buried it.
Decoded sentence: Je l'ai enterrée.

rand_pick: [2478]
-
Input sentence: I'm selfish.
Decoded sentence: Je suis si patient.

rand_pick: [3839]
-
Input sentence: I'm homesick.
Decoded sentence: Je suis en train de travailler.

rand_pick: [3797]
-
Input sentence: I'm a genius.
Decoded sentence: Je suis en train de travailler.

rand_pick: [6769]
-
Input sentence: We're wealthy.
Decoded sentence: Nous sommes riches.

rand_pick: [4933]
-
Input sentence: Don't give up.
Decoded sentence: Ne laissez pas tomber.

